In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# My first data set in Kaggle
Although the intention of the orginal poster of this dataset hoped to glean some price prediction in the rental market in Dublin, I believe this data will not support this.

However, I will be using it to practice some basic cleaning up processes.

## This is what is in this data set:


In [ ]:
# The full data set. Not very complete and not sure anything can be predicted from this. But it is a worthwhile exercise sanitising it ...
df = pd.read_csv('../input/predicting-dublin-rental-daftie/daft_v_2.csv')
df.head()

A couple of columns are contain information that is not helpful in the context of a broad view of the information. Street "Address" is perhaps not as usefull as 'Eircode' or even the more traditonal postal code like '6', '4', '7', etc.

Longitude and latitude may come in handy for geo location, but postal code would be sufficient for me.

Description does not contain information that add qualitive value to results, as well as ID.

## A smaller data frame ...

In [ ]:
# remove unecessary information that doesn't add to a fair assesment
small = df.drop(['address','description', 'longitude', 'latitude','ID'], axis=1)
small

The 'price' column is in a format that is readable to consumers, but not in a format for computing. The column needs to contain only intergers.

In [ ]:
#difficult to chart or analyse a string, as the dtypes reveals the price to be ...
small.dtypes

To maintain the distinction between the two frequencies in price of rent, I added a new column with boolean values for the presence of 'Per week' in the price column.


In [ ]:
small['freq'] = small['price'].str.contains('Per week')
small

Then I changed those boolean values and labeled them either 'weekly' or 'monthly'.

In [ ]:
# Change those booleans to string either 'weekly' or 'monthly'

small['freq'].replace({True:'weekly',False:'monthly'}, inplace=True)
small

Then I was able to safely remove the sub-strings 'Per week', 'Per month', and the '€' symbol, leaving behind only numbers.

However,these are still a <class: Object> type so they will need to be converted to intergers.

In [ ]:
# Now it's time to clean up the strings to leave only numbers behind, 
#and replacing the original column with a cleaned version of itself
small['price'] = small['price'].apply(lambda x: x.replace('€', '').replace('Per month', '').replace('Per week','').strip())
small

now prices are intergers:

In [ ]:
# removing commas and changing type of values under 'price'
small['price'] = small['price'].apply(lambda x: x.replace(',','')).astype(int)
small

In [ ]:
import seaborn as sns
%matplotlib inline

## Outliers
It is obvious from the first entry in this data set that there are outliers. Where are they?

In [ ]:
sns.boxplot(x='freq', y='price', data=small)

Monthly rates have the most extreme variance in outliying prices.


In [ ]:
count_w = sum(small['price'][small['freq']== 'weekly'].value_counts())
max_w = small['price'][small['freq']== 'weekly'].max()
count_m = sum(small['price'][small['freq']== 'monthly'].value_counts())
max_m = small['price'][small['freq']== 'monthly'].max()

print(f'There are {count_w} entries for weekly rents. The highest price is {max_w} (which would be equivalent to {round(max_w*52/12)} monthly)')
print(f'There are {count_m} monthly prices. The highest price is {max_m}')

In [ ]:
wk_mean = round(small['price'][small['freq']== 'weekly'].mean())
mt_mean = round(small['price'][small['freq']== 'monthly'].mean())

print(f'Weekly average price including outliers = {wk_mean}\nMonthly average price including outliers = {mt_mean}')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sns.displot(small['price'][small['freq']== 'weekly'])
plt.title("Weekly Prices Distribution")

In [ ]:
    sns.displot(small['price'][small['freq']== 'monthly'], bins=30)
    plt.title("Monthly Prices Distribution")

In [ ]:
# This is copied from another website. I don't know this function yet.
# this works with a series, so I create one similarly to the way I found a mean above:
w = small['price'][small['freq']== 'weekly']
m = small['price'][small['freq']== 'monthly']

#now find the range to work with:
w = round(w[w.between(w.quantile(.15), w.quantile(.85))].mean()) # without outliers
m = round(m[m.between(m.quantile(.15), m.quantile(.85))].mean()) # without outliers

print(f'Weekly mean average without outliers = {w}\nMonthly average without outliers = {m}')

### What does the charts look like when the outliers are removed?